<a href="https://colab.research.google.com/github/lpscby/MachineLearning101/blob/google_colab_receive/BY_ML101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第一课: Tech with Tim - Linear Regression
课程内容:使用UCL的[Student Performance数据集](https://archive.ics.uci.edu/ml/datasets/Student+Performance)，使用线性回归模型进行拟合和分数预测,并检查预测效果。

首先,上传本地文档(代码对话框方式，也可直接使用网页操作上传)

注意每次重新连接Google Colab时，都需重新上传文件。

In [ ]:
#上传本地文档
from google.colab import files

uploaded = files.upload()

第一步,从文件中读取数据,检查是否正确取出各项参数，熟悉pandas dataframe的操作方法。

In [ ]:
#读取CSV文档信息,并做检查,熟悉data操作方法
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from matplotlib import style
import pickle

style.use("ggplot")

data = pd.read_csv("/content/student-mat-comma.csv", sep = ",")

#print('Watch the data type: ', type(data))
#此时data数据类型是pandas.DataFrame
#print('Watch the data head: ', '\n', data.head())
#print(data.columns)
#print(data.index)

由于使用的是线性回归数学模型进行拟合，因此这里切分出来使用的参数都是数据。
除了Tim选择的G1, G2, G3, studytime, failures, absences以外,也可以考虑选择:age, Medu, Fedu, traveltime, famrel, freetime, goout, Dalc, Walc, health.

In [ ]:
#对data进行切分,便于下一步输入模型进行拟合

data = data[["G1", "G2", "G3", "studytime", "failures", "absences", "freetime", "goout"]]
data = shuffle(data)

print(data.head())

对抽取好的数据进行Label,选出需要预测的Label，并用sklearn的train_test_split函数进行数据集划分。

In [ ]:
predict = "G3"

x = np.array(data.drop([predict], 1))
#print(type(x))
#此时x的数据类型是numpy.ndarray

y = np.array(data[predict])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = 0.1)
#print(x_train)



利用linear regression模型进行学习和预测.为了找到更好的模型,循环训练20次,将最好的一次记入pickle(对Google Colab不适用,因为pickle文件会被清理掉).

*pickle提供了一个简单的持久化功能。可以将对象以文件的形式存放在磁盘上。

pickle模块只能在python中使用，python中几乎所有的数据类型（列表，字典，集合，类等）都可以用pickle来序列化，pickle序列化后的数据，可读性差，人一般无法识别。

pickle.dump(obj, file[, protocol])
　　序列化对象，并将结果数据流写入到文件对象中。参数protocol是序列化模式，默认值为0，表示以文本的形式序列化。protocol的值还可以是1或2，表示以二进制的形式序列化。

pickle.load(file)
　　反序列化对象。将文件中的数据解析为一个Python对象。

In [ ]:
best = 0
for _ in range(20):
  x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = 0.1)
  linear = linear_model.LinearRegression()
  
  linear.fit(x_train, y_train)
  acc = linear.score(x_test, y_test)
  print('The test predict accuracy score is: ', acc)
  if acc > best:
    best = acc
    with open("studentgrades.pickle", "wb") as f:
      pickle.dump(linear, f)

pickle_in = open("studentgrades.pickle", "rb")
linear = pickle.load(pickle_in)
#print('Coefficient: \n', linear.coef_)
#print('Intercept: \n', linear.intercept_)
print("The highest predict accuracy score is: %f" % best)

predictions = linear.predict(x_test)

for i in range(len(predictions)):
  print(predictions[i], x_test[i], y_test[i])


画图

In [ ]:
plot = "health"
plt.scatter(data[plot], data["G3"])
plt.legend(loc=4)
plt.xlabel(plot)
plt.ylabel("Final Grade")
plt.show()

# 第二课: Tech with Tim - KNN

课程内容: 使用UCL的[Car Data数据集](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)，使用KNN(K Nearest Neighbour)模型进行拟合和预测,并检查预测效果。

首先进行数据整理。

In [1]:
import sklearn
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing

data = pd.read_csv("car.data")
#print(data.head())

由于car data文件中有很多文字性信息，因此利用preprocessing这个方法将文字信息转换为整数。

In [2]:
le = preprocessing.LabelEncoder() # preprocessing.LabelEncoder是一个对象

buying = le.fit_transform(list(data["buying"])) #le还是一个空对象，将buying这一列转换为list，再赋给le对象进行转换
maint = le.fit_transform(list(data["maint"]))
door = le.fit_transform(list(data["door"]))
persons = le.fit_transform(list(data["persons"]))
lug_boot = le.fit_transform(list(data["lug_boot"]))
safety = le.fit_transform(list(data["safety"]))
cls = le.fit_transform(list(data["class"]))

predict = "class"

X = list(zip(buying, maint, door, persons, lug_boot, safety))
y = list(cls)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1)

#print(x_train, y_test)

然后可以按照Linear Regression同样的方法，构建KNN的模型，用训练数据集训练模型，然后给出预测并进行验证。

对KNN模型的构建参数以及方法的详细说明可以见[KNN模型的说明文档。](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
model = KNeighborsClassifier(n_neighbors = 9)
model.fit(x_train, y_train)
acc = model.score(x_test, y_test)
print(acc)

predicted = model.predict(x_test)
names = ["unacc", "acc", "good", "vgood"]

for num in range(len(predicted)):
  print('Predicted: %s; Actual: %s.' % (names[predicted[num]], names[y_test[num]]))

# 第三课 Tech with Tim - SVM

与Linear Regresstion和KNN示例中不一样，这一次SVM模型的例子中将用到SKLearn中自带的数据集。这些数据集中已经进行了分类，同时自带可以调用的方法，方便检查和处理数据。

具体而言，此次用到的是breast_cancer数据集。既然没有单独的csv文件供事先检查数据的features和labels，那么就需要利用SKLearn数据集中自带的methods来动态检查。

In [12]:
import sklearn
from sklearn import svm
from sklearn import datasets
from sklearn import metrics

cancer = datasets.load_breast_cancer()
print("Features: ", cancer.feature_names)
print("Labels: ", cancer.target_names)

Features:  ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Labels:  ['malignant' 'benign']


接下来可以按照一般流程进行数据处理了。

In [13]:
X = cancer.data #这个方法是导出所有features
y = cancer.target #这个方法是导出所有labels

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.2)

处理好训练集和预测集数据以后，开始构建模型、训练模型、输出预测并检验结果。

[SVM的官方说明文档](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm)

In [16]:
clf = svm.SVC() #在这一步注意选择kernel和正则化参数
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print(acc)

0.9649122807017544
